## iarchitect > data.py

In [1]:
import numpy as np
import pandas as pd

def get_data():
    """Method to get the tuile from csv files"""
    path = '../iarchitect/data/'
    
    # Import description csv as df_tuile
    csv_tuile = 'Tableau des caractéristiques - Caractéristiques'
    df_tuile = pd.read_csv(path + csv_tuile + '.csv')
    df_tuile.columns = df_tuile.head(1).values.tolist()[0]
    df_tuile = df_tuile.iloc[1: , :]

    # Import nemesis csv as df_nemesis
    csv_nemesis = 'Tableau des compatibilités - Matrice'
    df_nemesis = pd.read_csv(path + csv_nemesis + '.csv')
    df_nemesis.rename(columns={'Unnamed: 0':'Categorie'}, inplace=True)
    
    return df_tuile, df_nemesis
    
def clean_data(df_tuile, df_nemesis):
    """Clean df_tuile and df_nemesis and merge the two data frames"""
    ### DATA CLEANING in df_nemesis
    df_nemesis.drop_duplicates(inplace=True)

    df_nemesis = df_nemesis.set_index('Categorie')

    # Replace all Nan and positive values with 0
    df_nemesis = df_nemesis.fillna(0)
    df_nemesis = df_nemesis.mask(df_nemesis > 0, 0)

    # Update all negative values to 1
    df_nemesis = df_nemesis.mask(df_nemesis < 0, 1)

    # Reset the index and create Categorie_id
    df_nemesis = df_nemesis.reset_index()
    df_nemesis['Categorie_id'] = df_nemesis.index

    # Create dictionaries for Categories
    cat_dict = df_nemesis[['Categorie']].to_dict()['Categorie']
    cat_dict = {value : key for (key, value) in cat_dict.items()}

    # data selection
    df_nemesis = df_nemesis.drop(columns = ['Categorie', '& +20 espèces'])

    
    ### DATA CLEANING in df_tuile
    df_tuile.drop_duplicates(inplace=True)

    # Remove seasonality by excluding 'Nom' with numbers 
    df_tuile = df_tuile[~df_tuile['Nom'].str.contains('\\d', regex=True)]

    # Reset the index and create Nom_id
    df_tuile = df_tuile.reset_index()
    df_tuile['Nom_id'] = df_tuile.index

    # Replace Categorie with Categorie_id
    df_tuile = df_tuile.replace({"Categorie": cat_dict})
    df_tuile = df_tuile.rename(columns={'Categorie':'Categorie_id'})

    # Create dictionaries for Name
    name_dict = df_tuile[['Nom']].to_dict()['Nom']
    name_dict = {value : key for (key, value) in name_dict.items()}

    # Data selection
    df_tuile = df_tuile[['Nom_id', 'Profit','Rendement (kg/pied)','Popularité', 'Categorie_id']]

    
    ### MERGE DATA FRAMES
    df_tuile = df_tuile.merge(df_nemesis, how='left', on='Categorie_id')
    df_tuile = df_tuile.drop(columns = 'Categorie_id')
    df_tuile = df_tuile.fillna(0)

    # Return a numpy array
    tuile = df_tuile.to_numpy()
    return tuile, df_tuile, df_nemesis

if __name__ == '__main__':
    df_tuile, df_nemesis = get_data()
    tuile, df_tuile_clean, df_nemesis_clean = clean_data(df_tuile, df_nemesis)

In [2]:
df_tuile_clean.head()

,Nom_id,Profit,Rendement (kg/pied),Popularité,Ail blanc,Ail rose,Ail violet,Artichaut,Aubergine,Betterave,...,Ciboulette,Menthe,Persil,Cassis,Fraises,Framboisier,Groseiller,Myrtillier,Ronces,Rhubarbe
0,0,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
tuile

array([[0, '2', '1', ..., 0.0, 0.0, 0.0],
       [1, '2', '1', ..., 0.0, 0.0, 0.0],
       [2, '2', '1', ..., 0.0, 0.0, 0.0],
       ...,
       [63, '2', '1', ..., 0.0, 0.0, 0.0],
       [64, '2', '1', ..., 0.0, 0.0, 0.0],
       [65, '2', '1', ..., 0.0, 0.0, 0.0]], dtype=object)

,Nom_id,Profit,Rendement (kg/pied),Popularité,Ail blanc,Ail rose,Ail violet,Artichaut,Aubergine,Betterave,...,Ciboulette,Menthe,Persil,Cassis,Fraises,Framboisier,Groseiller,Myrtillier,Ronces,Rhubarbe
0,0,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,61,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
62,62,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
63,63,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
64,64,2,1,10,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
class Tuile:
    def __init__(self, action):
        # veg : price, productivity, capacity
        self.SCORE = {1:[10,1,1], 2:[8,1,1], 3:[5,1,1], 4:[2,1,1], 5:[6,1,1]}, 
        self.NEMESIS = {1:[], 2:[1,5], 3:[], 4:[3], 5:[]}
        self.rewards = {'PENALTY_NEMESIS':-5}

## Neighbours

In [ ]:
    def affinity(self): 

        x = action_[0]
        y = action_[1]
        
        if y-1 >=0:
            north = self._state[x, y-1]
        north = None
        if x+1 >=0 and y-1 >=0:
            north_east = self._state[x+1, y-1]
        north_east = None
        if x+1 >=0:
            east = self._state[x+1, y]
        east = None
        if x+1 >=0 and y+1 >=0:
            south_east = self._state[x+1, y+1]
        south_east = None
        if y+1 >= 0:
            south = self._state[x, y+1]
        south = None
        if x-1 >=0 and y+1 >=0:
            south_west = self._state[x-1, y+1]  
        south_west = None
        if x-1 >=0:
            west = self._state[x-1, y]
        west = None
        if x-1 >=0 and y-1 >=0:
            north_west = self._state[x-1, y-1]
        north_west = None
                                          
        for nem in self.NEMESIS[action_[2]]:
            if nem in [north, north_east, east, south_east,
                       south, south_west, west, north_west]:
                reward = self.rewards['penalty_nemesis']
        
    def revenue(self):
        for key, value in self.SCORE.items() :
            if key == action_[2]:
                reward = self.rewards['new_value'] + value[0]
                